In [1]:
# third-party
import pandas as pd
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

### Activity-specific only RP analysis

In [3]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [4]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,SNB,MAG,"0.17 , 0.28","8.43 , 13.26","0.18 , 0.33","7.8 , 11.88","0.16 , 0.32","3.4 , 5.43"
1,SNB,PZT,"0.37 , 0.5","15.62 , 22.19","0.38 , 0.53","12.94 , 18.33","0.29 , 0.45","4.64 , 7.01"
2,SGB,MAG,"0.8 , 0.64","17.01 , 14.64","0.69 , 0.66","11.85 , 11.34","0.61 , 0.63","5.22 , 4.99"
3,SGB,PZT,"1.66 , 1.35","36.94 , 30.78","1.7 , 1.34","27.81 , 21.73","0.93 , 1.24","7.7 , 10.92"
4,MIXB,MAG,"0.19 , 0.19","7.51 , 7.52","0.18 , 0.17","6.25 , 5.74","0.2 , 0.21","3.51 , 3.28"
5,MIXB,PZT,"0.56 , 0.72","17.27 , 22.19","0.64 , 0.87","16.58 , 25.04","0.65 , 0.96","8.14 , 11.9"
6,STNB,MAG,"0.16 , 0.24","7.3 , 9.85","0.19 , 0.29","7.46 , 10.8","0.13 , 0.22","2.48 , 2.69"
7,STNB,PZT,"0.29 , 0.44","10.53 , 15.44","0.35 , 0.56","10.87 , 18.98","0.26 , 0.48","4.02 , 7.54"
8,MCH,MAG,"0.31 , 0.36","14.31 , 14.77","0.33 , 0.36","15.89 , 15.8","0.33 , 0.42","7.13 , 8.29"
9,MCH,PZT,"0.63 , 0.6","28.61 , 24.6","0.67 , 0.64","30.65 , 27.9","0.62 , 0.65","13.03 , 13.55"


In [5]:
# mag = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True) 
# pzt = performance_all_participants[performance_all_participants["Sensor"] == "PZT"].loc[:,["SSE Ti", "SSE Te", "SSE Tb"]].reset_index(drop=True)

# a = mag / pzt 
# a["Activity"] = performance_all_participants[performance_all_participants["Sensor"] == "MAG"].reset_index(drop=True)["Activity"]
# a

### Overall RP analysis

In [6]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [7]:
get_breath_parameters_performance(overview_all, target=None)

,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,MAG,"0.29 , 0.36","13.9 , 15.84","0.3 , 0.37","13.82 , 15.31","0.26 , 0.37","5.45 , 7.07"
1,PZT,"0.57 , 0.6","25.76 , 23.85","0.58 , 0.61","26.05 , 24.14","0.48 , 0.6","10.17 , 11.68"


In [8]:
rp_detection_all = get_breath_parameters_performance(overview, target="both")
rp_detection_all

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:163: RuntimeWarning:

invalid value encountered in divide

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:198: RuntimeWarning:

invalid value encountered in scalar divide

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in sca

,ID,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,7OYX,SNB,MAG,"0.1 , 0.08","5.6 , 6.11","0.12 , 0.08","5.16 , 3.4","0.07 , 0.07","1.5 , 1.4"
1,7OYX,SNB,PZT,"0.66 , 0.17","39.78 , 27.51","0.61 , 0.27","17.72 , 13.41","0.69 , 0.63","4.42 , 2.33"
2,7OYX,SGB,MAG,"0.58 , 0.48","13.31 , 10.1","0.76 , 0.6","12.95 , 10.75","0.22 , 0.15","2.1 , 1.34"
3,7OYX,SGB,PZT,"3.62 , 0.85","41.73 , 27.69","2.86 , 2.33","40.92 , 38.11","0.73 , 0.0","1.9 , 0.0"
4,7OYX,MIXB,MAG,"0.1 , 0.08","5.26 , 2.66","0.11 , 0.15","4.48 , 3.69","0.11 , 0.06","3.11 , 1.53"
...,...,...,...,...,...,...,...,...,...
473,D4GQ,UAR,PZT,"0.31 , 0.04","1.97 , 0.77","0.26 , 0.17","3.55 , 1.48","0.17 , 0.05","0.74 , 0.27"
474,D4GQ,SE,MAG,"0.64 , 0.7","22.36 , 21.75","0.63 , 0.83","17.3 , 18.04","0.67 , 0.84","10.85 , 12.58"
475,D4GQ,SE,PZT,"0.8 , 0.71","8.07 , 3.08","0.51 , 0.7","14.73 , 20.78","0.23 , 0.13","1.8 , 1.15"
476,D4GQ,SS,MAG,"0.79 , 0.81","27.04 , 29.78","0.76 , 0.74","26.69 , 19.63","0.58 , 0.51","10.14 , 10.07"


In [9]:
bp_mag = rp_detection_all[rp_detection_all["Sensor"]=="MAG"]
bp_pzt = rp_detection_all[rp_detection_all["Sensor"]=="PZT"]

In [10]:
metric = "MAE Tb (s)"
normality_test(bp_mag[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], sensor=metric, type="All data")
normality_test(bp_pzt[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], sensor=metric, type="All data")

In [11]:
for metric in ["MAE Tb (s)", "MAE Ti (s)", "MAE Te (s)"]:
    print(f'{metric}: {scipy.stats.wilcoxon(bp_mag[metric].str.split(" , ", expand=True).astype(float).iloc[:,0], bp_pzt[metric].str.split(" , ", expand=True).astype(float).iloc[:,0])}')



MAE Tb (s): WilcoxonResult(statistic=nan, pvalue=nan)
MAE Ti (s): WilcoxonResult(statistic=nan, pvalue=nan)
MAE Te (s): WilcoxonResult(statistic=nan, pvalue=nan)


In [12]:
wilc = pd.DataFrame(columns=["Activity", "Metric", "Statistic", "p-value"])

for metric in ["MAE Tb (s)", "MAE Ti (s)", "MAE Te (s)"]:
    for activity in bp_mag["Activity"].unique():
        new_entry = {}
        new_entry["Activity"] = activity
        new_entry["Metric"] = metric
        new_entry["Statistic"], new_entry["p-value"] = scipy.stats.wilcoxon(bp_mag[bp_mag["Activity"]==activity][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values, bp_pzt[bp_pzt["Activity"]==activity][metric].str.split(" , ", expand=True).astype(float).iloc[:,0].values)
        wilc.loc[len(wilc)] = new_entry
        
wilc

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching to normal approximation.

/Users/anasofiacc/dev/M-RESP/.venv/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning:

Exact p-value calculation does not work if there are zeros. Switching t

,Activity,Metric,Statistic,p-value
0,SNB,MAE Tb (s),44.5,0.231201
1,SGB,MAE Tb (s),NaN,NaN
2,MIXB,MAE Tb (s),10.0,0.007630
3,STNB,MAE Tb (s),25.0,0.046826
4,MCH,MAE Tb (s),13.0,0.002686
5,SQT,MAE Tb (s),19.0,0.009186
6,AAL,MAE Tb (s),NaN,NaN
7,AAR,MAE Tb (s),22.0,0.030908
8,ALL,MAE Tb (s),13.0,0.002686
9,ALR,MAE Tb (s),NaN,NaN


In [13]:
### Bias and variability analysis

In [14]:
for metric in ["tI (s)", "tE (s)", "tB (s)"]:
    for sensor in ["MAG", "PZT"]:
        bland_altman_plot(overview_all[sensor][metric], overview_all[sensor][metric.split(" ")[0] + " airflow " + metric.split(" ")[1]], metric=metric.split(" ")[0], sensor=sensor)
